In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 네이버 댓글 스크래핑: network API 활용

## 패키지(라이브러리) 불러오기

In [2]:
import pandas as pd
import json
import re
import requests
from tqdm import tqdm

## 하나의 뉴스에 대해 네이버 url 이용해 댓글 수집하기


*   해당 url에 접속해 검사 창 열기
*   Network > JS(JavaScript) 확인: 댓글 더보기를 클릭하면 이 곳에 파일 생성  
*   web_naver_list_jsonp.json?~~ 파일에서 Response 확인: 수집하려는 댓글 포함
*   web_naver_list_jsonp.json?~~ 파일에서 Headers 확인 -> Request URL 확보


In [3]:
story_url='https://n.news.naver.com/mnews/article/comment/056/0011701465?sid=104'

header = {                      # API에 담아 전송할 기본 정보
    "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0",
    "referer": story_url
}

vs=['userIdNo', 'contents', 'userName', 'regTime', 'sympathyCount', 'antipathyCount']   # 수집할 댓글과 관련 정보들의 변수(이후 json 데이터에서 key에 해당)

objectId=re.findall('(\d{3}/\d{9,11})', story_url)[0].replace('/','%2C') # '언론사ID/기사ID'(urld의 뒷부분)
page = 1

# 검사 창에서 확보한 Request URL 구성
request_url = f'''https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&pool=cbox5&lang=ko&country=KR&objectId=news{objectId}&page={str(page)}
&pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=20&sort=old&includeAllStatus=true'''

response = requests.get(request_url, headers=header)       # 웹에 접속 요청
jsonData = re.findall("{.*}", response.text)[0]            # 전송 받은 문자열에서 json 부분({} 형태) 추출
jsonData = json.loads(jsonData)                            # json 타입으로 전환

comment_n = int(jsonData['result']['count']['comment'])  # 댓글 갯수 확인 # 댓글 갯수. 스포츠 연예는 댓글 없음.
if comment_n == 0:
    print(story_url+': 댓글이 없습니다')
else:
    pass

next = jsonData['result']['morePage']['next']               # 더보기를 통해 이동할 다음 페이지 정보 추출

result=[]
for i in jsonData['result']['commentList']:            # json 데이터에서 댓글 100개 관련 부분 확보
    try:
        result.append([i[v] for v in vs])              # 각 댓글과 관련 정보를 key를 통한 호출로 수집
    except:
        continue

    while True:                 # 댓글 더보기를 지속하며 댓글 전수 수집을 위한 반복문
        request_url2 = request_url + f'&moreParam.next={next}'

        response = requests.get(request_url2, headers=header)
        jsonData = re.findall("{.*}", response.text)[0]
        jsonData = json.loads(jsonData)

        comment_n = int(jsonData['result']['count']['comment'])   # 댓글 갯수.
        try:
            next = jsonData['result']['morePage']['next']
        except:
            pass

        for i in jsonData['result']['commentList']:
            try:
                result.append([i[v] for v in vs])
            except:
                continue

        if comment_n <= ((page) * 100):                # 댓글 수에 따라 페이지 지속 중단 여부 판단
            break
        else:
            page += 1

results=pd.DataFrame(result, columns=vs)             # 데이터프레임으로 정리. content 내용 없는 경우는 삭제된 것

## 함수화 getComments

In [4]:
import pandas as pd
import json
import re
import requests

def getComments(story_url):
    header = {                      # API에 담아 전송할 기본 정보
        "User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0",
        "referer": story_url
    }
    vs=['userIdNo', 'contents', 'userName', 'regTime', 'sympathyCount', 'antipathyCount']   # 수집할 댓글과 관련 정보들의 변수(이후 json 데이터에서 key에 해당)
    objectId=re.findall('(\d{3}/\d{9,11})', story_url)[0].replace('/','%2C') # '언론사ID/기사ID'(urld의 뒷부분)
    page = 1
    # 검사 창에서 확보한 Request URL 구성
    request_url = f'''https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&pool=cbox5&lang=ko&country=KR&objectId=news{objectId}&page={str(page)}
    &pageSize=100&indexSize=10&groupId=&listType=OBJECT&pageType=more&initialize=true&followSize=5&userType=&useAltSort=true&replyPageSize=20&sort=old&includeAllStatus=true'''

    response = requests.get(request_url, headers=header)       # 웹에 접속 요청
    jsonData = re.findall("{.*}", response.text)[0]            # 전송 받은 문자열에서 json 부분({} 형태) 추출
    jsonData = json.loads(jsonData)                            # json 타입으로 전환

    comment_n = int(jsonData['result']['count']['comment'])  # 댓글 갯수 확인 # 댓글 갯수. 스포츠 연예는 댓글 없음.
    if comment_n == 0:
        print(story_url+': 댓글이 없습니다')
    else:
        pass

    next = jsonData['result']['morePage']['next']               # 더보기를 통해 이동할 다음 페이지 정보 추출
    result=[]
    for i in jsonData['result']['commentList']:            # json 데이터에서 댓글 100개 관련 부분 확보
        try:
            result.append([i[v] for v in vs])              # 각 댓글과 관련 정보를 key를 통한 호출로 수집
        except:
            continue

        while True:                 # 댓글 더보기를 지속하며 댓글 전수 수집을 위한 반복문
            request_url2 = request_url + f'&moreParam.next={next}'

            response = requests.get(request_url2, headers=header)
            jsonData = re.findall("{.*}", response.text)[0]
            jsonData = json.loads(jsonData)

            comment_n = int(jsonData['result']['count']['comment'])   # 댓글 갯수.
            try:
                next = jsonData['result']['morePage']['next']
            except:
                pass

            for i in jsonData['result']['commentList']:
                try:
                    result.append([i[v] for v in vs])
                except:
                    continue

            if comment_n <= ((page) * 100):                # 댓글 수에 따라 페이지 지속 중단 여부 판단
                break
            else:
                page += 1
    return result

In [ ]:
story_url='https://n.news.naver.com/article/052/0002020026'

result=getComments(story_url)
vs=['userIdNo', 'contents', 'userName', 'regTime', 'sympathyCount', 'antipathyCount']
df_result=pd.DataFrame(result, columns=vs)

## 복수의 뉴스 데이터(네이버 url 포함)에서 댓글 추출하기

*   네이버 뉴스 데이터 불러오기(url 포함)


In [9]:
import pandas as pd
#df = pd.read_excel("/content/drive/MyDrive/2024_1_class/KBS_이란.xlsx")  # 네이버뉴스 수집한 파일(url 포함) 불러오기

df = pd.read_excel('/content/drive/MyDrive/518/navernews_5등_20220501_20240531.xlsx')
df['naver_url']=df['url'].apply(lambda x: 1 if 'naver' in x else 0)
df=df[df['naver_url']==1]
df.info()

2      1
5      1
7      1
8      1
9      1
      ..
641    1
642    1
643    1
645    1
646    1
Name: naver_url, Length: 247, dtype: int64



*   함수를 이용해 for문 작동


In [ ]:
results=[]
for story_url in tqdm(df['url']): # 해당 파일에서 네이버 뉴스 url에 해당하는 변수 'n_url'의 값에 대해 반복문 실시
    try:
        result=getComments(story_url)
        result1=[[story_url,a,b,c,d,e,f] for a,b,c,d,e,f in result]   # 각 댓글마다 story_url 표기
        results.extend(result1)
    except:
        pass

vs=['userIdNo', 'contents', 'userName', 'regTime', 'sympathyCount', 'antipathyCount']
results_final=pd.DataFrame(results, columns=['url']+vs)